In [1]:
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())

# Tutorial 7 - Case Statements in Fluent Syntax

Case statements are expressions much like `if`/`else` statements in other programming languages. They consist of a chain of `WHEN` (condition) and `THEN` (value) pairs ending in an `ELSE` that covers the value of the default case. 
For example
```
CASE
  WHEN some_value > 5 and some_value < 10
  THEN 'large'
  WHEN some_value > 0 and some_value < 5
  THEN 'medium'
  ELSE 'out of bounds'
END
```

In `lumipy` these are built by calling the `.when()` method on a table and supplying a condition to it (column expression that resolves to a bool). Next you chain a then by calling the `.then()` method and supplying a python literal value or a column expression. This pattern is repeated until you've covered the cases and ends by setting the default by calling `.otherwise()` and optionally supplying a python literal value or a column expression. If no input is given here the default case will just evaluate to `null`. 
```
case = table.when(
    (table.some_value > 5) & (table.some_value < 10)
).then("large").when(
    (table.some_value > 0) & (table.some_value < 5)
).then("medium").otherwise("out of bounds")
```

In [2]:
portfolios = atlas.lusid_portfolio()
holdings = atlas.lusid_portfolio_holding()
properties = atlas.lusid_property()

## Example 1

Add a new column (`InstrumentTheme`) with a value that depends on whether the portfolio code has 'bond', 'equity' or 'swap' in it. 

Note that because this is not an original column of the table but a derived one it must be supplied as a keyword arg in the `select()` method. This is how the case expression gets a name. 

In [3]:
case = portfolios.when(
    portfolios.portfolio_code.contains('equit')
).then("Equity").when(
    portfolios.portfolio_code.contains('bond')
).then("Debt").when(
    portfolios.portfolio_code.contains('swap')
).then("Swap").otherwise('Other')

In [4]:
pf_qry = portfolios.select(
    '^',
    InstrumentTheme=case
).where(
    portfolios.portfolio_scope == 'Finbourne-Examples'
)
pf_df = pf_qry.go()
pf_df

,BaseCurrency,PortfolioCode,PortfolioScope,PortfolioType,InstrumentTheme
0,GBP,UK-Swaps-aggressive,Finbourne-Examples,Transaction,Swap
1,GBP,Global-Equity,Finbourne-Examples,Transaction,Equity
2,GBP,UK-Equities,Finbourne-Examples,Transaction,Equity
3,USD,US-Corporate-Bond,Finbourne-Examples,Transaction,Debt
4,USD,US-Treasury-Bond,Finbourne-Examples,Transaction,Debt


In [5]:
# print the underlying SQL for inspection
pf_qry.print_sql()


select
  [BaseCurrency], [PortfolioCode], [PortfolioScope], [PortfolioType], (  
  CASE
      WHEN [PortfolioCode] like '%equit%'
        THEN 'Equity'
      WHEN [PortfolioCode] like '%bond%'
        THEN 'Debt'
      WHEN [PortfolioCode] like '%swap%'
        THEN 'Swap'
    ELSE 'Other'
  END
) as [InstrumentTheme] 
from
  Lusid.Portfolio 
where
  [PortfolioScope] = 'Finbourne-Examples'


## Example 2

This next example shows how you can use a column expression in the `then()` method and how a case statement can be built up inside a for loop. 

We have a dictionary of currencies and their conversion rate to dollars. This case will return the holdings cost amount multiplied by the rate given the currency. If the currency rate isn't available it'll default to `null`.

In [6]:
ccy_rates_to_usd = {
    'USD': 1.0,
    'GBP': 1.38,
    'EUR': 1.19,
    'JPY': 0.0091
}

conv_case = holdings
for ccy, rate in ccy_rates_to_usd.items():
    
    conv_case = conv_case.when(
        holdings.cost_currency == ccy
    ).then(
        rate*holdings.cost_amount
    )

conv_case = conv_case.otherwise() # default case (null)

In [7]:
hld_qry = holdings.select(
    holdings.lusid_instrument_id,
    holdings.portfolio_code,
    holdings.units,
    holdings.cost_currency,     
    holdings.cost_amount,   
    CostAmountUSD=conv_case    
).where(
    holdings.portfolio_scope == 'Finbourne-Examples'
)
hld_df = hld_qry.go()
hld_df.head()

,LusidInstrumentId,PortfolioCode,Units,CostCurrency,CostAmount,CostAmountUSD
0,LUID_JTQY6QFI,Global-Equity,1.200000e+07,USD,2.390760e+09,2.390760e+09
1,LUID_J4G8D0DV,Global-Equity,8.390274e+06,EUR,1.329858e+09,1.582531e+09
2,LUID_TMYK4GZV,Global-Equity,1.098658e+07,JPY,7.197106e+10,6.549366e+08
3,LUID_WW55WKWV,Global-Equity,-8.191471e+06,USD,-1.092849e+09,-1.092849e+09
4,LUID_7M3OX0BB,Global-Equity,1.085971e+07,USD,1.314567e+09,1.314567e+09


In [8]:
hld_qry.print_sql()


select
  [LusidInstrumentId], [PortfolioCode], [Units], [CostCurrency], [CostAmount], (  
  CASE
      WHEN [CostCurrency] = 'USD'
        THEN [CostAmount] * 1.0
      WHEN [CostCurrency] = 'GBP'
        THEN [CostAmount] * 1.38
      WHEN [CostCurrency] = 'EUR'
        THEN [CostAmount] * 1.19
      WHEN [CostCurrency] = 'JPY'
        THEN [CostAmount] * 0.0091
    ELSE null
  END
) as [CostAmountUSD] 
from
  Lusid.Portfolio.Holding 
where
  [PortfolioScope] = 'Finbourne-Examples'
